In [140]:
import pandas as pd
import numpy as np
import torch
from torch.nn import Module, Conv2d, Linear, Softmax, ReLU, CrossEntropyLoss, Sequential, BatchNorm2d, MaxPool2d, Dropout
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim import Adam
from torchvision import transforms
import os
from glob import glob
from tqdm import tqdm
import pickle as pkl
from PIL import Image

In [152]:
corrupted_img = transforms.functional.rgb_to_grayscale(Image.open("data/sat_data/sat_data/images/15.jpg"))

In [173]:
class SatelliteImages(Dataset):
    def __init__(self, data_path, image_dir, indices_file):
        super(SatelliteImages, self).__init__()
        self.data = []
        self.labels = []
        self.ids = []

        df = pd.read_csv(data_path)
        indices = np.load(indices_file)
        images = glob(os.path.join(image_dir, "*.jpg"))
        convert_tensor = transforms.ToTensor()
        normalize = transforms.Normalize([0], [1])
        resize = transforms.Resize((256, 256))

        for image in tqdm(images):
            try:
                idx = int(os.path.basename(image).split(".")[0])
                if idx in indices:
                    df_entry = df.iloc[idx]
                    #print(type(df_entry.roof_shape))
                    if type(df_entry.roof_shape) == str:
                        label = 1 if df_entry.roof_shape == "flat" else 0
                        img = transforms.functional.rgb_to_grayscale(Image.open(image))
                        # if img == corrupted_img:
                        #     continue
                        #print(image)
                        self.data.append(normalize(convert_tensor(resize(img))))
                        self.labels.append(label)
                        self.ids.append(idx)
            except Exception:
                continue


    def __len__(self):
        return len(self.labels)

    def __getitem__(self, item):
        return self.data[item], self.labels[item]

In [172]:
satellite_data_train = SatelliteImages("data/processed_data.csv", "data/sat_data/sat_data/images", "data/sat_data/sat_data/train_indices.npy")
with open("data/roof_dataset_train.pkl", "wb") as fh:
    pkl.dump(satellite_data_train, fh)


100%|██████████| 15000/15000 [00:17<00:00, 863.75it/s] 


In [175]:
np.random.seed(42)

zero_inds = np.where(np.array(satellite_data_train.labels) == 0)[0]
one_inds = np.where(np.array(satellite_data_train.labels) == 1)[0]
num_nonzero = np.count_nonzero(np.array(satellite_data_train.labels))
train_inds = list(zero_inds[:num_nonzero])
train_inds.extend(one_inds)
np.random.shuffle(train_inds)
len(train_inds)

374

In [176]:
satellite_data_test = SatelliteImages("data/processed_data.csv", "data/sat_data/sat_data/images", "data/sat_data/sat_data/val_indices.npy")
with open("data/roof_dataset_test.pkl", "wb") as fh:
    pkl.dump(satellite_data_test, fh)


100%|██████████| 15000/15000 [00:08<00:00, 1721.23it/s]


In [177]:
zero_inds = np.where(np.array(satellite_data_test.labels) == 0)[0]
one_inds = np.where(np.array(satellite_data_test.labels) == 1)[0]
num_nonzero = np.count_nonzero(np.array(satellite_data_test.labels))
test_inds = list(zero_inds[:num_nonzero])
test_inds.extend(one_inds)
len(test_inds)

172

In [36]:
with open("data/roof_dataset_train.pkl", "rb") as fh:
    satellite_data_train = pkl.load(fh)
with open("data/roof_dataset_test.pkl", "rb") as fh:
    satellite_data_test = pkl.load(fh)

torch.Size([1, 256, 256])

In [280]:
class SatelliteCNN(Module):
    def __init__(self, n_classes=2):
        super(SatelliteCNN, self).__init__()
        self.conv = Conv2d(in_channels=1, out_channels=1, kernel_size=12, stride=2)
        self.relu = ReLU()
        self.linear = Linear(in_features=15129, out_features=n_classes)

    def forward(self, batch):
        batch_conv = self.relu(self.conv(batch))
        batch_flatten = batch_conv.squeeze(1).flatten(start_dim=1)
        #print(batch_flatten.size())
        return self.linear(batch_flatten)

In [281]:
def train_epoch(model, device, dataloader, criterion, optimizer):
    model.train()
    correct_train = 0
    train_loss = 0
    for j, (batch, batch_labels) in enumerate(dataloader):
        batch = batch.to(device)
        batch_labels = batch_labels.to(device)

        optimizer.zero_grad()
        pred = model(batch)
        loss = criterion(pred, batch_labels)
        loss.backward()
        optimizer.step()

        train_loss += loss * batch.size(0)
        correct_train += (pred.argmax(dim=1) == batch_labels).sum().item()

    return train_loss, correct_train

def evaluate_epoch(model, device, dataloader, criterion):
    model.eval()
    correct_val = 0
    evaluate_loss = 0
    with torch.no_grad():
        for batch, batch_labels in dataloader:
            batch = batch.to(device)
            batch_labels = batch_labels.to(device)

            pred = model(batch)
            loss = criterion(pred, batch_labels)

            evaluate_loss += loss.item() * batch.size(0)
            correct_val += (pred.argmax(dim=1) == batch_labels).sum().item()

    return evaluate_loss, correct_val

In [287]:
def train_model(train_set, learning_rate, n_epochs, batch_size, results_dir):
    os.makedirs(results_dir, exist_ok=True)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Device: {device}")

    criterion = CrossEntropyLoss().to(device)

    update_progress_bar = tqdm(total=n_epochs, desc="train_loss: {np.nan:7.5f}, train_acc: {np.nan:7.5f}, validation_loss: {np.nan:7.5f} validation_acc: {np.nan:7.5f}", position=0)

    val_size = int(.2 * len(train_inds))
    train_idx, val_idx = random_split(train_inds, [len(train_inds) - val_size, val_size])
    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)
    # val_size = int(.2 * len(train_set))
    # train_idx, val_idx = random_split(train_set, [len(train_set) - val_size, val_size])
    train_loader = DataLoader(train_set, batch_size=batch_size, sampler=train_sampler)
    val_loader = DataLoader(train_set, batch_size=batch_size, sampler=val_sampler)

    model = SatelliteCNN(n_classes=2)
    model = model.to(device)

    optimizer = Adam(model.parameters(), lr=learning_rate)

    best_val_acc = 0

    for epoch in range(n_epochs):
        train_loss, train_correct = train_epoch(model, device, train_loader, criterion, optimizer)
        val_loss, val_correct = evaluate_epoch(model, device, val_loader, criterion)

        train_loss = train_loss / len(train_loader.sampler)
        train_acc = train_correct / len(train_loader.sampler) * 100
        val_loss = val_loss / len(val_loader.sampler)
        val_acc = val_correct / len(val_loader.sampler) * 100

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save({
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "loss": train_loss
            }, os.path.join(results_dir, f"best_model.pt"))

        update_progress_bar.set_description(f"train_loss: {train_loss:7.5f}, train_acc: {train_acc:7.5f}, validation_loss: {val_loss:7.5f}, validation_acc: {val_acc:7.5f}", refresh=True)
        update_progress_bar.update()

    print("Finished Training.")
    update_progress_bar.close()

In [288]:
train_model(satellite_data_train, learning_rate=1e-3, n_epochs=20, batch_size=32, results_dir="results_0")

Device: cuda


train_loss: 0.01184, train_acc: 100.00000, validation_loss: 1.23623, validation_acc: 43.24324: 100%|██████████| 20/20 [00:02<00:00,  9.21it/s]           

Finished Training.


In [289]:
def test_model(model_path):
    best_model = SatelliteCNN(n_classes=2)
    checkpoint = torch.load(os.path.join(model_path))
    best_model.load_state_dict(checkpoint["model_state_dict"])
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Device: {device}")
    best_model.to(device)
    test_sampler = SubsetRandomSampler(test_inds)
    test_loader = DataLoader(satellite_data_test, batch_size=32, sampler=test_sampler)

    y_pred = []
    y_true = []
    correct_val = 0

    best_model.eval()
    with torch.no_grad():
        for batch, batch_labels in test_loader:
            batch = batch.to(device)
            batch_labels = batch_labels.to(device)

            output = best_model(batch)
            correct_val += (output.argmax(dim=1) == batch_labels).sum().item()

            output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
            y_pred.extend(output)

            labels = batch_labels.data.cpu().numpy()
            y_true.extend(labels)

        accuracy = correct_val / len(test_inds)
        print(accuracy)

In [290]:
test_model("results_0/best_model.pt")

Device: cuda
0.5058139534883721
